# ResNet && CIFAR10/CIFAR100
We test ResNet classifier.
First init some basic environment.

In [1]:
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

One time installation of needed lib from requirement.txt and creating dir 

In [ ]:
!pip install -r requirements.txt
!mkdir data

    100% |████████████████████████████████| 61kB 73.3MB/s ta 0:00:01
    100% |████████████████████████████████| 10.4MB 93.0MB/s ta 0:00:01         | 1.9MB 58.3MB/s eta 0:00:01��████████▊              | 5.8MB 91.0MB/s eta 0:00:01��█████████████| 10.4MB 95.4MB/s eta 0:00:01
    100% |████████████████████████████████| 7.0MB 97.4MB/s eta 0:00:01     | 1.7MB 97.5MB/s eta 0:00:01
    100% |████████████████████████████████| 378kB 98.4MB/s eta 0:00:01MB/s eta 0:00:01
    100% |████████████████████████████████| 686kB 7.4MB/s ta 0:00:011
    100% |████████████████████████████████| 51kB 68.3MB/s ta 0:00:01
    100% |████████████████████████████████| 51kB 78.6MB/s ta 0:00:01


# resnet20/56/110 model with CIFAR10 / CIFAR100


In [ ]:
import torch.optim as optim
import torch.nn as nn
import time
from tnt_solver_ import *
from models.resnet_ import resnet20, resnet56, resnet110
from dataset.dataset_dowloader_ import *

cifar10_dowloader()
cifar100_dowloader()
model = resnet20()
print(model)

# Start learning 

## TODO
+ Stohastic gradient decrease? May be adam?
+ lr?
+ epoch?

In [ ]:
%%time
import torch.optim as optim
import torch.nn.functional as F
opt = optim.SGD(model.parameters(), lr=1e-1, momentum=0.9, weight_decay=1e-4, nesterov=False)
loss_fn = F.cross_entropy

lr_scheduler= optim.lr_scheduler.MultiStepLR(opt, milestones=[91, 137], gamma=0.1)
history = main(model, opt, epoch=180, loss_fn=loss_fn, lr_scheduler=lr_scheduler)

In [ ]:
plot_history(history)

# GREAT TODO
+ Test each modification 3 times for the average results
+ Solve some of todos above
+ Play with lr, epochs, STD
+ Graphics
+ Think about the best way to analyse all results
+ May be to play with VGG?
+ Why with classic startup options there is over learning after 90th epoch???

Control each model learning startup to re-define all weights. (maybe) 

# Pandas

In [5]:
!pip3 install pandas

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
     |████████████████████████████████| 10.4MB 19.0MB/s eta 0:00:01██▊                            | 1.2MB 19.0MB/s eta 0:00:01      | 5.1MB 19.0MB/s eta 0:00:01█████████████████████████     | 8.8MB 19.0MB/s eta 0:00:01
     |████████████████████████████████| 512kB 63.0MB/s eta 0:00:01
     |████████████████████████████████| 20.0MB 37.0MB/s eta 0:00:01                    | 266kB 37.0MB/s eta 0:00:01�███▏                       | 5.1MB 37.0MB/s eta 0:00:01ta 0:00:01 12.8MB 37.0MB/s eta 0:00:01


In [20]:
import pandas as pd
import numpy as np

d = { "lrate" : [0.01, 0.001, 0.0001],
  "epochs": [10, 20, 30],
  "optimizer": ["Adam", "SGD", "Adadelta"]
}

# params = {
#     'lr': [0.01, 0.02],
#     'max_epochs': [10, 20],
# }
params = {
    'epoch': [10, 20],
}

In [21]:
df = pd.DataFrame(d)
df

,lrate,epochs,optimizer
0,0.0100,10,Adam
1,0.0010,20,SGD
2,0.0001,30,Adadelta


In [22]:
def train_model(lrate, epochs, optimizer):
    print(lrate, epochs, optimizer)
    return 'GOVNO'

In [23]:
models = []
for i, (lrate, epochs, optimizer) in df.iterrows():
    models.append(train_model(lrate, epochs, optimizer))
print(models)

0.01 10 Adam
0.001 20 SGD
0.0001 30 Adadelta
['GOVNO', 'GOVNO', 'GOVNO']


In [24]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV

In [27]:
iris = datasets.load_iris()
# parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
# svc = svm.SVC()
# clf = GridSearchCV(svc, parameters)
# clf.fit(iris.data, iris.target)

clf = GridSearchCV(main, params)
clf.fit(iris.data, iris.target)


TypeError: estimator should be an estimator implementing 'fit' method, <function main at 0x7f9372d30ea0> was passed

In [ ]:
clf.cv_results_